In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.src.layers import TextVectorization
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv("tripadvisor_hotel_reviews.csv")
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [3]:
df['Rating'].unique()

array([4, 2, 3, 5, 1])

In [4]:
#split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df["Review"], df["Rating"], test_size=0.2, random_state=42)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [5]:
# import nltk
# nltk.download('stopwords')

In [6]:
import string
import re

stopwords = set(stopwords.words("english"))

def text_preprocessing(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    for word in stopwords:
        if word[0] == "'":
            text = tf.strings.regex_replace(text, rf"{word}\b", "")
        else:
            text = tf.strings.regex_replace(text, rf"\b{word}\b", "")

    text = tf.strings.regex_replace(text, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', "")
    return text

train_sentence = [text_preprocessing(sentence) for sentence in X_train]
test_sentence = [text_preprocessing(sentence) for sentence in X_test]

In [7]:
train_sentence

[<tf.Tensor: shape=(), dtype=string, numpy=b'outstanding cleanliness value location wanted stay central location london luna simone hotels fell search area checked reviews trip advisor decided book certainly  dissapointed location ideal short walk transit important sight seeing locations absolutely cleanest hotels stayed breakfast good served management gives chance talk owners management staff outstanding friendly assist site seeing plans  additional charge fact probably save money got london bus tour tickets hotel vendor rate thinking trip uk europe luna simone london hotel staying going london reccomend luna simone  '>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'cheap cheerful hey  sure fuss picked hotel price got paid dont wrong  palace clean tidy warm honest expecting grander check little paid think  rooms basic livable location  bad free public ferry leaves central station leaves  metres botel cant crying lazy arses thinks use mini bus collection returning home little worse wea

In [8]:
vectorize = tf.keras.layers.TextVectorization(max_tokens = 15000)
vectorize.adapt(X_train)
vectorized_x_train = vectorize(train_sentence)
vectorized_x_test = vectorize(test_sentence)

In [9]:
print(vectorized_x_test)

tf.Tensor(
[[  47    2  965 ...    0    0    0]
 [ 270 1467   16 ...    0    0    0]
 [ 144    2    7 ...    0    0    0]
 ...
 [ 101  216   48 ...    0    0    0]
 [   5  219  860 ...    0    0    0]
 [  93    9 1385 ...    0    0    0]], shape=(4099, 1653), dtype=int64)


In [10]:
train_ds = tf.data.Dataset.from_tensor_slices((vectorized_x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((vectorized_x_test, y_test))

In [11]:
train_ds = train_ds.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE).batch(64)
test_ds = test_ds.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE).batch(64)

In [12]:
model = tf.keras.Sequential([])
model.add(tf.keras.layers.Input(shape=(15000,)))
model.add(tf.keras.layers.Embedding(15000, 128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(6, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 15000, 128)     │     1,920,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 15000, 128)     │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,138,470 (8.16 MB)

 Trainable params: 2,138,470 (8.16 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [17]:
model.compile(loss="categorical_crossentropy", metrics=['accuracy'], optimizer='rmsprop')

In [18]:
model.fit(train_ds, epochs=4, validation_data=test_ds)

Epoch 1/4
257/257 ━━━━━━━━━━━━━━━━━━━━ 1597s 6s/step - accuracy: 0.5212 - loss: 1.0957 - val_accuracy: 0.4565 - val_loss: 1.1241
Epoch 2/4
257/257 ━━━━━━━━━━━━━━━━━━━━ 1648s 6s/step - accuracy: 0.6271 - loss: 0.8430 - val_accuracy: 0.5836 - val_loss: 1.0158
Epoch 3/4
257/257 ━━━━━━━━━━━━━━━━━━━━ 1890s 7s/step - accuracy: 0.6680 - loss: 0.7565 - val_accuracy: 0.6380 - val_loss: 0.8796
Epoch 4/4
257/257 ━━━━━━━━━━━━━━━━━━━━ 2288s 9s/step - accuracy: 0.7073 - loss: 0.6880 - val_accuracy: 0.6136 - val_loss: 1.0673
